In [21]:
#Perform imports
import random

import numpy as np
import torch
from torch import optim
from tqdm import tqdm

import gymnasium as gym


from stable_baselines3 import A2C

from stable_baselines3.common.logger import configure


import matplotlib.pyplot as plt


device = 'cpu'# 'cuda' #  #doing cpu as A2C with MlpPolicy (rather than CNNpolicy) in stablebaseline is faster on CPU, and the meta gradinet beign faster on GPU (even if it is) is not *that* much faster - it is about two(ish) times slower overall based on one run each with two meta iterations, so better on cpu in this case
torch.set_default_device(device)

In [22]:
model_save_path = "saved_models\\29May_MuJoCoStandardTest_3"

In [23]:
#Make meta-environment
from fabian.envs.cartpole_gymn import MuJoCo_ActionOffsets
env = MuJoCo_ActionOffsets(gymnasium_env_name='InvertedPendulum-v5', 
                               offset_variance=0.1,
                               )
env.normalise_actions()
env.add_recorder(model_save_path, "trainig_")

In [24]:
##Set up for meta learning

#Hyperparameters
adapt_lr =  7e-4
meta_lr = 0.0005 
meta_iterations = 500#1250
adapt_timesteps = 32*4 #for this enviornment, each episode is exactly 32 timesteps, so multiple of 32 means full number of eps experienced for each task
tasks_per_loop = 40#60
adapt_visualisations = 15

vis_timesteps = meta_iterations//adapt_visualisations #denominator is number of visualisations we want
if vis_timesteps == 0:
    vis_timesteps = 1 



#Make meta-policy and meta-optimiser
meta_agent = A2C("MlpPolicy", env, verbose=0, learning_rate=adapt_lr, device=device,
                    )
                 #meta_learning=True, 
                # M=1,adapt_timesteps=4*32, eval_timesteps=100) #we train the meta_agent to do well at adapting to new envs (i.e. meta learning) in our env distribution
#meta_opt = optim.Adam(meta_agent.policy.parameters(), lr=meta_lr)

#Logging variables
meta_losses = []
meta_rets = []
best_meta_ret = None
best_meta_ret_it = -1

In [25]:
env.task 

0.13111005537008683

In [26]:
meta_agent.learn(total_timesteps=1_000_000, progress_bar=True)

Output()

In [27]:
env.task 

0.13111005537008683

In [28]:
env.set_recording(True)
meta_agent.evaluate_policy(total_timesteps=2000, progress_bar=True, policy=meta_agent.policy)

(tensor(-1.0723e-07, grad_fn=<DivBackward0>), tensor(100.0000))

In [29]:
env.task 

0.13111005537008683

In [10]:
env.close()